In [1]:
library(knitr)
library(sqldf)
library(ggplot2)
library(mongolite)
library(ggplot2)
library(gridExtra)
library(dplyr)
library(tidyr)
library(jsonlite)
library(lubridate)
library(gridExtra)
library(GGally)
library(scales)
library(memisc)
 library(sp)
library(rgeos)
library(rgdal)
library(maptools)
library(RColorBrewer)
library(maps)
library(grid)
library(cluster)
library(StatMatch)
library(fastcluster)
library(factoextra)
library(stringr)
library(googleVis)
library(RMySQL)
library(data.table)
library(dbplyr)
library(DBI)
library(httr)
library(scanstatistics)
library(purrr)
library(magrittr)
library(rsatscan)
library(stringi)

Loading required package: gsubfn
Loading required package: proto
Loading required package: RSQLite

Attaching package: 'dplyr'

The following object is masked from 'package:gridExtra':

    combine

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date


Attaching package: 'GGally'

The following object is masked from 'package:dplyr':

    nasa

Loading required package: lattice
Loading required package: MASS

Attaching package: 'MASS'

The following object is masked from 'package:dplyr':

    select


Attaching package: 'memisc'

The following object is masked from 'package:scales':

    percent

The following object is masked from 'package:lubridate':

    is.interval

The following objects are masked from 'package:dplyr':

    collect, recode, rename, syms

The following objec

In [34]:
ferg_latlon_n <- read.csv("C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Features_Demo\\ferg_latlon_n.csv")

In [35]:
ferg_latlon_n$X <- NULL
str(ferg_latlon_n)

'data.frame':	9349 obs. of  5 variables:
 $ location: Factor w/ 4293 levels "ABBEVILLE SC",..: 1 2 2 3 4 5 5 6 7 7 ...
 $ time    : int  6 2 6 6 6 4 6 6 2 3 ...
 $ count1  : int  2 1 2 1 7 4 7 4 3 3 ...
 $ min_lon : num  -82.4 -76.2 -76.2 -79.4 -98.5 ...
 $ min_lat : num  34.2 39.5 39.5 35.1 45.5 ...


In [40]:
ferg_latlon_n2 <- read.csv("C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Features_Demo\\ferg_latlon_n.csv")

In [41]:
ferg_latlon_n2$X <-NULL
str(ferg_latlon_n2)

'data.frame':	24198 obs. of  5 variables:
 $ location: Factor w/ 5692 levels "ABBEVILLE SC",..: 1 2 2 2 2 3 3 4 4 4 ...
 $ time    : int  6 2 6 10 11 6 10 6 7 10 ...
 $ count   : int  2 1 2 1 1 1 1 7 2 6 ...
 $ lon     : num  -82.4 -76.2 -76.2 -76.2 -76.2 ...
 $ lat     : num  34.2 39.5 39.5 39.5 39.5 ...


In [21]:
nrow(unique(ferg_latlon_n2[c("lon", "lat", "time")]))

[1] 24198

In [28]:
nrow(ferg_latlon_n2 %>% group_by(lon, lat) %>%
                        summarise(first.row = first(location)))
                   #      arrange(location), 20)

[1] 5692

###### Get population of cities

In [30]:
pop_cities <- read.csv("../Preprocessing/Files/Demographics/usstates_placetype.csv")

In [31]:
pop_cities_2015 <- pop_cities[c("City_code", "Popestimate2015", "Place_type")]

In [32]:
pop_cities_2015 <- pop_cities_2015 %>% dplyr::rename("pop2015" = "Popestimate2015") %>%
                    
                  as.data.frame()

In [33]:
pop_cities_2015 <- pop_cities_2015 %>% filter(Place_type %in% c("city", 
                                                                "city/town", 
                                                                "municipality", "town", 
                                                                "village", "township")) 

In [42]:
pop_cities_2015$ City_code <- gsub(",", "", toupper(as.character(pop_cities_2015$City_code)))
ferg_latlon_n2$location <- as.character(ferg_latlon_n2$location)

In [37]:
#head(pop_cities_2015) 
subset(pop_cities_2015, City_code == "NEW YORK NY")

,City_code,pop2015,Place_type
49804,NEW YORK NY,8582459,city


In [44]:
ferg_latlon_cities_count <- ferg_latlon_n2 %>% 
                            dplyr::select(one_of(c("location", "count"))) %>%
                            inner_join(pop_cities_2015, by = c("location" = "City_code")) %>%
                            group_by(location) %>%
                                    summarise(counts = sum(count),
                                             population = sum(pop2015)) %>%
                            
                            as.data.frame()

In [45]:
nrow(unique(ferg_latlon_cities_count))

[1] 5378

In [41]:
head(ferg_latlon_cities_count)

location,counts,population
ABBEVILLE SC,4,10228
ABERDEEN MD,6,62068
ABERDEEN NC,2,14572
ABERDEEN SD,35,112285
ABERDEEN WA,22,64904
ABILENE KS,16,26016


In [42]:
##### Get unique location, counts and population

In [46]:
city_count_pop <- unique(ferg_latlon_cities_count[c("location", "counts", "population")])

write.csv(city_count_pop, "C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\city_count_pop.csv")

Correlation between total_counts and population. 

In [47]:
cor.test(ferg_latlon_cities_count$counts, ferg_latlon_cities_count$population)


	Pearson's product-moment correlation

data:  ferg_latlon_cities_count$counts and ferg_latlon_cities_count$population
t = 51.602, df = 5376, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.5573855 0.5931420
sample estimates:
      cor 
0.5755387 


If income done either upper, middle, working or $ value depends on results

Age <- numerical. Race Categorical. Urban, rural, surburban.

###### Load Age data

In [48]:
age <- read.csv("../Preprocessing/Files/Demographics/acs_2015_as2_data.csv", 
                  stringsAsFactors = FALSE)

In [54]:
age_cols <- age[c(1:4,6,8,seq(28, 118, 6), 178)]

In [48]:
str(age_cols)

'data.frame':	29322 obs. of  23 variables:
 $ GEO.id           : chr  "Id" "1600000US0100100" "1600000US0100124" "1600000US0100460" ...
 $ GEO.id2          : chr  "Id2" "100100" "100124" "100460" ...
 $ GEO.display.label: chr  "Geography" "Abanda CDP, Alabama" "Abbeville city, Alabama" "Adamsville city, Alabama" ...
 $ HC01_EST_VC01    : chr  "Total; Estimate; Total population" "52" "2646" "4454" ...
 $ HC02_EST_VC01    : chr  "Male; Estimate; Total population" "11" "1292" "2269" ...
 $ HC03_EST_VC01    : chr  "Female; Estimate; Total population" "41" "1354" "2185" ...
 $ HC01_EST_VC06    : chr  "Total; Estimate; AGE - 15 to 19 years" "0" "4.3" "7.6" ...
 $ HC01_EST_VC07    : chr  "Total; Estimate; AGE - 20 to 24 years" "0" "5.9" "7.2" ...
 $ HC01_EST_VC08    : chr  "Total; Estimate; AGE - 25 to 29 years" "0" "3.7" "6.6" ...
 $ HC01_EST_VC09    : chr  "Total; Estimate; AGE - 30 to 34 years" "0" "4.4" "5.8" ...
 $ HC01_EST_VC10    : chr  "Total; Estimate; AGE - 35 to 39 years" "0" "3.7"

In [49]:
head(age_cols)

GEO.id,GEO.id2,GEO.display.label,HC01_EST_VC01,HC02_EST_VC01,HC03_EST_VC01,HC01_EST_VC06,HC01_EST_VC07,HC01_EST_VC08,HC01_EST_VC09,...,HC01_EST_VC13,HC01_EST_VC14,HC01_EST_VC15,HC01_EST_VC16,HC01_EST_VC17,HC01_EST_VC18,HC01_EST_VC19,HC01_EST_VC20,HC01_EST_VC23,HC01_EST_VC35
Id,Id2,Geography,Total; Estimate; Total population,Male; Estimate; Total population,Female; Estimate; Total population,Total; Estimate; AGE - 15 to 19 years,Total; Estimate; AGE - 20 to 24 years,Total; Estimate; AGE - 25 to 29 years,Total; Estimate; AGE - 30 to 34 years,...,Total; Estimate; AGE - 50 to 54 years,Total; Estimate; AGE - 55 to 59 years,Total; Estimate; AGE - 60 to 64 years,Total; Estimate; AGE - 65 to 69 years,Total; Estimate; AGE - 70 to 74 years,Total; Estimate; AGE - 75 to 79 years,Total; Estimate; AGE - 80 to 84 years,Total; Estimate; AGE - 85 years and over,Total; Estimate; SELECTED AGE CATEGORIES - 5 to 14 years,Total; Estimate; SUMMARY INDICATORS - Median age (years)
1600000US0100100,100100,"Abanda CDP, Alabama",52,11,41,0,0,0,0,...,0,23.1,55.8,0,21.2,0,0,0,0,60.5
1600000US0100124,100124,"Abbeville city, Alabama",2646,1292,1354,4.3,5.9,3.7,4.4,...,7.7,10.1,7.9,7,7.8,6.5,3,4.7,13,52.5
1600000US0100460,100460,"Adamsville city, Alabama",4454,2269,2185,7.6,7.2,6.6,5.8,...,8.1,8,5.3,6.4,4.3,3.1,2.8,0.2,12.3,39.6
1600000US0100484,100484,"Addison town, Alabama",682,331,351,7,4.7,7.2,3.8,...,3.1,10,3.1,8.4,7,3.5,4.8,1.6,12.2,42.5
1600000US0100676,100676,"Akron town, Alabama",293,143,150,13.7,10.6,3.8,1.7,...,15.4,9.6,5.5,4.1,3.8,3.1,1.4,1,3.8,43.6


In [ ]:
### Get column names

In [55]:
colnames(age_cols) <- age_cols %>% dplyr::slice(1)

In [56]:
age_cols <- age_cols[2:nrow(age_cols), ]

In [ ]:
### Rename some columns

In [57]:
colnames(age_cols) <- trimws(gsub("Total; Estimate;", "", colnames(age_cols)), "both")
colnames(age_cols) <- trimws(gsub("(;|-| )", "", colnames(age_cols)), "both")

In [53]:
head(subset(age_cols, ))

,Id,Id2,Geography,Totalpopulation,MaleEstimateTotalpopulation,FemaleEstimateTotalpopulation,AGE15to19years,AGE20to24years,AGE25to29years,AGE30to34years,...,AGE50to54years,AGE55to59years,AGE60to64years,AGE65to69years,AGE70to74years,AGE75to79years,AGE80to84years,AGE85yearsandover,SELECTEDAGECATEGORIES5to14years,SUMMARYINDICATORSMedianage(years)
2,1600000US0100100,100100,"Abanda CDP, Alabama",52,11,41,0,0,0,0,...,0,23.1,55.8,0,21.2,0,0,0,0,60.5
3,1600000US0100124,100124,"Abbeville city, Alabama",2646,1292,1354,4.3,5.9,3.7,4.4,...,7.7,10.1,7.9,7,7.8,6.5,3,4.7,13,52.5
4,1600000US0100460,100460,"Adamsville city, Alabama",4454,2269,2185,7.6,7.2,6.6,5.8,...,8.1,8,5.3,6.4,4.3,3.1,2.8,0.2,12.3,39.6
5,1600000US0100484,100484,"Addison town, Alabama",682,331,351,7,4.7,7.2,3.8,...,3.1,10,3.1,8.4,7,3.5,4.8,1.6,12.2,42.5
6,1600000US0100676,100676,"Akron town, Alabama",293,143,150,13.7,10.6,3.8,1.7,...,15.4,9.6,5.5,4.1,3.8,3.1,1.4,1,3.8,43.6
7,1600000US0100820,100820,"Alabaster city, Alabama",31905,15590,16315,7.5,5.7,5.4,6.6,...,7.9,5.3,4.7,3.9,2.4,2,1.3,1.1,16.7,36.6


age_cols[ , 4:ncol(age_cols)] <- sapply(age_cols[ , 4:ncol(age_cols)], as.numeric)

age_cols[which(is.na(age_cols[4:ncol(age_cols)))] <- 0

In [ ]:
#### Get key columns

In [64]:
age_cols2 <- age_cols %>% dplyr::select(one_of("Id", "Id2", "Geography", "Totalpopulation", 
                                               "MaleEstimateTotalpopulation", "FemaleEstimateTotalpopulation",
                                              "SUMMARYINDICATORSMedianage(years)")) %>%

                                #    mutate(pop_under_34 = rowSums(as.numeric(as.matrix(.[7:10])))) %>%
                                                                            
                                                     
                                                  #   per_pop_under_34 = round(pop_under_34/"Total population" * 100,2)
                                                    # %>%
                                    as.data.frame()

age_cols2 <- age_cols %>% dplyr::select(one_of(c("Id", "Id2", "Geography", "Totalpopulation", 
                                               "MaleEstimateTotalpopulation", "FemaleEstimateTotalpopulation",
                                             "AGE15to19years", "AGE20to24years", "AGE25to29years", 
                                              "AGE30to34years", "SUMMARYINDICATORSMedianage(years)"))) %>%
                            as.data.frame()

In [ ]:
### Convert to numbers

In [65]:
age_cols2[c("Totalpopulation", "MaleEstimateTotalpopulation", 
            "FemaleEstimateTotalpopulation",
                                   "SUMMARYINDICATORSMedianage(years)")] <- 
sapply(age_cols2[c("Totalpopulation", "MaleEstimateTotalpopulation", 
                   "FemaleEstimateTotalpopulation",
                                              "SUMMARYINDICATORSMedianage(years)")], 
       as.numeric)

Warning message in lapply(X = X, FUN = FUN, ...):
"NAs introduced by coercion"

In [ ]:
summary(age_cols2)

###### Replace NAs in MedianAge

In [66]:

age_cols2 <- age_cols2 %>% dplyr::rename("MalePop" = "MaleEstimateTotalpopulation", 
                                         "FemalePop" = "FemaleEstimateTotalpopulation" ,
                           "MedianAge" = "SUMMARYINDICATORSMedianage(years)") %>%
                            as.data.frame()

In [67]:
age_cols2$MedianAge[which(is.na(age_cols2$MedianAge))] <- 0

In [ ]:
### % male and female population

In [68]:
age_cols2 <- age_cols2 %>% group_by(Id, Id2, Geography, Totalpopulation, 
                                    MalePop, FemalePop, MedianAge) %>%
                                    summarise(per_male = 
                                              round(MalePop/Totalpopulation * 100, 2),
                                             per_female = 
                                              round(FemalePop/Totalpopulation * 100, 
                                                    2)) %>%
                            as.data.frame()

In [69]:
colnames(age_cols2) <- tolower(colnames(age_cols2))

#### Load statistics about urban/rural populations

In [83]:
urban <- read.csv("../Preprocessing/Files/Demographics/acs_2015_ur3.csv", 
                  stringsAsFactors = FALSE)

In [84]:
urban_cols <- urban[c(3:7,10)]

In [ ]:
### Rename columns

In [85]:
colnames(urban_cols) <- tolower(urban_cols %>% dplyr::slice(1))

In [86]:
colnames(urban_cols) <- trimws(gsub(":", "", colnames(urban_cols)), "both")
urban_cols$total <- trimws(gsub("(\\d{1,})\\(r\\d{1,}\\)", "\\1", 
                                urban_cols$total), "both")

In [88]:
urban_cols[ , 4:ncol(urban_cols)] <- sapply(urban_cols[ , 4:ncol(urban_cols)], as.numeric)

In [ ]:
### Replace NAs

In [89]:
urban_cols$total[which(is.na(urban_cols$total))] <- 0
urban_cols$urban[which(is.na(urban_cols$urban))] <- 0
urban_cols$rural[which(is.na(urban_cols$rural))] <- 0

In [79]:
summary(urban_cols)

      id                id2             geography             total        
 Length:27164       Length:27164       Length:27164       Min.   :      0  
 Class :character   Class :character   Class :character   1st Qu.:    435  
 Mode  :character   Mode  :character   Mode  :character   Median :   1265  
                                                          Mean   :   8406  
                                                          3rd Qu.:   4524  
                                                          Max.   :8175133  
     urban             rural        
 Min.   :      0   Min.   :    0.0  
 1st Qu.:      0   1st Qu.:   42.0  
 Median :     80   Median :  255.5  
 Mean   :   7888   Mean   :  517.7  
 3rd Qu.:   4222   3rd Qu.:  667.0  
 Max.   :8175057   Max.   :39432.0  

In [80]:
tail(urban_cols)

,id,id2,geography,total,urban,rural
27159,1600000US5683040,5683040,"Wheatland town, Wyoming",3627,3355,272
27160,1600000US5683765,5683765,"Wilson CDP, Wyoming",1482,0,1482
27161,1600000US5684925,5684925,"Worland city, Wyoming",5487,5371,116
27162,1600000US5685015,5685015,"Wright town, Wyoming",1807,0,1807
27163,1600000US5686665,5686665,"Yoder town, Wyoming",151,0,151
27164,1600000US5686737,5686737,"Y-O Ranch CDP, Wyoming",195,0,195


In [91]:
urban_cols2 <- urban_cols %>% dplyr::rename("total_area" = "total") %>%
                                group_by(id, id2, geography, total_area, 
                                    urban, rural) %>%
                                    summarise(per_urban = round(urban/total_area * 100, 2),
                                             per_rural = round(rural/total_area * 100, 2)
                                             ) %>%
                                  #  mutate(isurban = ifelse(per_urban >= 70, 1, 0)) %>%
                            as.data.frame()

In [92]:
urban_cols2 <- subset(urban_cols2, id != "Id")

In [93]:
summary(urban_cols2)

      id                id2             geography           total_area     
 Length:27163       Length:27163       Length:27163       Min.   :    100  
 Class :character   Class :character   Class :character   1st Qu.:    435  
 Mode  :character   Mode  :character   Mode  :character   Median :   1265  
                                                          Mean   :   8406  
                                                          3rd Qu.:   4524  
                                                          Max.   :8175133  
     urban             rural           per_urban        per_rural     
 Min.   :      0   Min.   :    0.0   Min.   :  0.00   Min.   :  0.00  
 1st Qu.:      0   1st Qu.:   42.0   1st Qu.:  0.00   1st Qu.:  0.71  
 Median :     80   Median :  256.0   Median : 12.07   Median : 87.93  
 Mean   :   7888   Mean   :  517.8   Mean   : 46.44   Mean   : 53.56  
 3rd Qu.:   4223   3rd Qu.:  667.0   3rd Qu.: 99.29   3rd Qu.:100.00  
 Max.   :8175057   Max.   :39432.0   Max. 

###### Load income per capita demographics

In [94]:
income <- read.csv("../Preprocessing/Files/Demographics/acs_2015_inc_percapita_data.csv", 
                  stringsAsFactors = FALSE)

In [95]:
income_cols <- income[c(1:4, 6, 60, 62)]

In [96]:
head(income_cols,6)

GEO.id,GEO.id2,GEO.display.label,HC01_EST_VC02,HC02_EST_VC02,HC01_EST_VC20,HC02_EST_VC20
Id,Id2,Geography,Total; Estimate; All households,Mean income (dollars); Estimate; All households,Total; Estimate; PER CAPITA INCOME BY RACE AND HISPANIC OR LATINO ORIGIN - Total population,Mean income (dollars); Estimate; PER CAPITA INCOME BY RACE AND HISPANIC OR LATINO ORIGIN - Total population
1600000US0100100,100100,"Abanda CDP, Alabama",52,14867,52,14867
1600000US0100124,100124,"Abbeville city, Alabama",1056,44246,2646,19262
1600000US0100460,100460,"Adamsville city, Alabama",1704,56085,4454,22032
1600000US0100484,100484,"Addison town, Alabama",317,44483,682,21389
1600000US0100676,100676,"Akron town, Alabama",126,24006,293,11410


In [ ]:
### Rename columns

In [97]:
colnames(income_cols) <- tolower(income_cols %>% dplyr::slice(1))

In [98]:
income_cols <- income_cols[2:nrow(income_cols), ]

In [99]:
colnames(income_cols)[4:7] <- c("total_population_income_households", 
                                "mean_income_households", 
                                "total_population_individuals", "mean_income_per_capita")

In [100]:
income_cols[ , 4:ncol(income_cols)] <- sapply(income_cols[ , 4:ncol(income_cols)], as.numeric)

Warning message in lapply(X = X, FUN = FUN, ...):
"NAs introduced by coercion"Warning message in lapply(X = X, FUN = FUN, ...):
"NAs introduced by coercion"

In [101]:
summary(income_cols)

      id                id2             geography        
 Length:29321       Length:29321       Length:29321      
 Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character  
                                                         
                                                         
                                                         
                                                         
 total_population_income_households mean_income_households
 Min.   :      0                    Min.   :   713        
 1st Qu.:    128                    1st Qu.: 45065        
 Median :    416                    Median : 55275        
 Mean   :   2975                    Mean   : 63721        
 3rd Qu.:   1531                    3rd Qu.: 70891        
 Max.   :3113535                    Max.   :987277        
                                    NA's   :789           
 total_population_individuals mean_income_per_capita
 Min.   :  

In [102]:
income_cols$state <- as.character(trimws(str_split_fixed(income_cols$geography, ",", n = 2)[, 2], 
                                      "both"))

median_state <- income_cols %>% dplyr::select("state", "mean_income", "total_income",
                                              "mean_income_per_capita") %>%
                                    group_by(state) %>%
                                    summarise(med_inc = round(median(mean_income, na.rm = TRUE)),
                                              med_inc_capita_races = 
                                              round(median(mean_income_per_capita, na.rm = TRUE))) %>%
                            ungroup() %>%
                            as.data.frame()

In [11]:
head(income_cols)

,id,id2,geography,total_population_income_households,mean_income_households,total_population_individuals,mean_income_per_capita,state
2,1600000US0100100,100100,"Abanda CDP, Alabama",52,14867,52,14867,Alabama
3,1600000US0100124,100124,"Abbeville city, Alabama",1056,44246,2646,19262,Alabama
4,1600000US0100460,100460,"Adamsville city, Alabama",1704,56085,4454,22032,Alabama
5,1600000US0100484,100484,"Addison town, Alabama",317,44483,682,21389,Alabama
6,1600000US0100676,100676,"Akron town, Alabama",126,24006,293,11410,Alabama
7,1600000US0100820,100820,"Alabaster city, Alabama",10556,80353,31905,27382,Alabama


In [ ]:
### Replace missing income with mean and median

In [103]:
income_cols <- income_cols %>% 
               group_by(state, add = FALSE) %>%
               mutate(mean_income_households = ifelse(total_population_income_households == 
                                                      0, 0, 
                                            replace(mean_income_households, 
                                                    is.na(mean_income_households), 
                                           round(mean(mean_income_households, 
                                                      na.rm = TRUE))))) %>%
                mutate(mean_income_per_capita = ifelse(total_population_individuals == 
                                                       0, 0, 
                                                       replace(mean_income_per_capita,
                                                        is.na(mean_income_per_capita), 
                        round(median(mean_income_per_capita, na.rm = TRUE))))) %>%
                as.data.frame()

In [161]:
summary(income_cols)

      id                id2             geography        
 Length:29321       Length:29321       Length:29321      
 Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character  
                                                         
                                                         
                                                         
 total_population_income_households mean_income_households
 Min.   :      0                    Min.   :     0        
 1st Qu.:    128                    1st Qu.: 45040        
 Median :    416                    Median : 55260        
 Mean   :   2975                    Mean   : 63261        
 3rd Qu.:   1531                    3rd Qu.: 70761        
 Max.   :3113535                    Max.   :987277        
 total_population_individuals mean_income_per_capita    state          
 Min.   :      0              Min.   :     0         Length:29321      
 1st Qu.:    331              1st Qu.

In [104]:
nrow(income_cols)

[1] 29321

###### Load Household income demographics

In [109]:
house_income <- read.csv("../Preprocessing/Files/Demographics/acs_2015_inc_median_households_data.csv", 
                  stringsAsFactors = FALSE)

In [110]:
house_income_cols <- house_income[c(1:4, 6)]

In [314]:
head(house_income_cols)

,id,id2,geography,total_income,median_income
2,1600000US0100100,100100,"Abanda CDP, Alabama",52,11207
3,1600000US0100124,100124,"Abbeville city, Alabama",1056,25615
4,1600000US0100460,100460,"Adamsville city, Alabama",1704,42575
5,1600000US0100484,100484,"Addison town, Alabama",317,37083
6,1600000US0100676,100676,"Akron town, Alabama",126,21667
7,1600000US0100820,100820,"Alabaster city, Alabama",10556,71816


In [ ]:
### Rename columns

In [111]:
colnames(house_income_cols) <- tolower(house_income_cols %>% dplyr::slice(1))

In [112]:
house_income_cols <- house_income_cols[2:nrow(house_income_cols), ]

In [113]:
colnames(house_income_cols)[4:5] <- c("total_income", "median_income")

In [114]:
summary(house_income_cols)

      id                id2             geography         total_income      
 Length:29321       Length:29321       Length:29321       Length:29321      
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
 median_income     
 Length:29321      
 Class :character  
 Mode  :character  

In [116]:
house_income_cols[ , 4:ncol(house_income_cols)] <- 
                    sapply(house_income_cols[ , 4:ncol(house_income_cols)], as.numeric)

In [89]:
summary(house_income_cols)

      id                id2             geography          total_income    
 Length:29321       Length:29321       Length:29321       Min.   :      0  
 Class :character   Class :character   Class :character   1st Qu.:    128  
 Mode  :character   Mode  :character   Mode  :character   Median :    416  
                                                          Mean   :   2975  
                                                          3rd Qu.:   1531  
                                                          Max.   :3113535  
                                                                           
 median_income   
 Min.   :  4511  
 1st Qu.: 35625  
 Median : 45305  
 Mean   : 51177  
 3rd Qu.: 59482  
 Max.   :244083  
 NA's   :1937    

In [117]:
house_income_cols$state <- as.character(trimws(str_split_fixed(house_income_cols$geography, ",", n = 2)[, 2], 
                                      "both"))

In [ ]:
#### Replace missing data with median

In [118]:
house_income_cols <- house_income_cols %>%
                dplyr::rename("house_total_population" = "total_income", 
                              "house_median_income" = "median_income") %>%
                group_by(state, add = FALSE) %>%
                mutate(house_median_income = ifelse(house_total_population == 0, 0, 
                                            replace(house_median_income, 
                                                    is.na(house_median_income), 
                                           round(median(house_median_income, na.rm = TRUE))))) %>%
             #   mutate(mean_income_per_capita = ifelse(total_income == 0, 0, 
                                                     #  replace(mean_income_per_capita,
                                                     #   is.na(mean_income_per_capita), 
                       # round(median(mean_income_per_capita, na.rm = TRUE))))) %>%
                as.data.frame()

In [162]:
summary(house_income_cols)

      id                id2             geography        
 Length:29321       Length:29321       Length:29321      
 Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character  
                                                         
                                                         
                                                         
 house_total_population house_median_income    state          
 Min.   :      0        Min.   :     0      Length:29321      
 1st Qu.:    128        1st Qu.: 35898      Class :character  
 Median :    416        Median : 44874      Mode  :character  
 Mean   :   2975        Mean   : 50487                        
 3rd Qu.:   1531        3rd Qu.: 58125                        
 Max.   :3113535        Max.   :244083                        

###### Load race data

In [119]:
race <- read.csv("../Preprocessing/Files/Demographics/acs_2015_race_data.csv", 
                  stringsAsFactors = FALSE)

In [120]:
race_cols <- race[c(1:4, 6,8)]

In [121]:
head(race_cols,3)

GEO.id,GEO.id2,GEO.display.label,HD01_VD01,HD01_VD02,HD01_VD03
Id,Id2,Geography,Estimate; Total:,Estimate; Total: - White alone,Estimate; Total: - Black or African American alone
1600000US0100100,100100,"Abanda CDP, Alabama",52,40,12
1600000US0100124,100124,"Abbeville city, Alabama",2646,1396,1227


In [122]:
colnames(race_cols) <- tolower(race_cols %>% dplyr::slice(1))

In [123]:
race_cols <- race_cols[2:nrow(race_cols), ]

In [124]:
colnames(race_cols)[4:6] <- c("total_population", "white_population", "black_population")

In [125]:
summary(race_cols)

      id                id2             geography         total_population  
 Length:29321       Length:29321       Length:29321       Length:29321      
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
 white_population   black_population  
 Length:29321       Length:29321      
 Class :character   Class :character  
 Mode  :character   Mode  :character  

In [126]:
race_cols[ , 4:ncol(race_cols)] <- sapply(race_cols[ , 4:ncol(race_cols)], as.numeric)

In [127]:
summary(race_cols)

      id                id2             geography         total_population 
 Length:29321       Length:29321       Length:29321       Min.   :      0  
 Class :character   Class :character   Class :character   1st Qu.:    331  
 Mode  :character   Mode  :character   Mode  :character   Median :   1087  
                                                          Mean   :   8043  
                                                          3rd Qu.:   4090  
                                                          Max.   :8426743  
 white_population  black_population 
 Min.   :      0   Min.   :      0  
 1st Qu.:    265   1st Qu.:      0  
 Median :    901   Median :      7  
 Mean   :   5613   Mean   :   1135  
 3rd Qu.:   3263   3rd Qu.:    157  
 Max.   :3648214   Max.   :2065597  

In [ ]:
### Get state

In [128]:
race_cols$state <- as.character(trimws(str_split_fixed(race_cols$geography, ",", n = 2)[, 2], 
                                      "both"))

In [129]:
race_cols <- race_cols %>% dplyr::rename("total_population_race" = "total_population") %>%
                mutate(per_white = ifelse(total_population_race == 0, 
                                          0, round(white_population/total_population_race * 100, 2))) %>%
                mutate(per_black = ifelse(total_population_race == 0, 
                                          0, round(black_population/total_population_race * 100, 2))) %>%
                as.data.frame()

In [366]:
summary(race_cols)

      id                id2             geography         total_population 
 Length:29321       Length:29321       Length:29321       Min.   :      0  
 Class :character   Class :character   Class :character   1st Qu.:    331  
 Mode  :character   Mode  :character   Mode  :character   Median :   1087  
                                                          Mean   :   8043  
                                                          3rd Qu.:   4090  
                                                          Max.   :8426743  
 white_population  black_population     state             per_white     
 Min.   :      0   Min.   :      0   Length:29321       Min.   :  0.00  
 1st Qu.:    265   1st Qu.:      0   Class :character   1st Qu.: 79.05  
 Median :    901   Median :      7   Mode  :character   Median : 92.69  
 Mean   :   5613   Mean   :   1135                      Mean   : 83.31  
 3rd Qu.:   3263   3rd Qu.:    157                      3rd Qu.: 97.71  
 Max.   :3648214   Max.   :206

###### Load employment demographics

In [132]:
employ <- read.csv("../Preprocessing/Files/Demographics/acs_2015_emp_data.csv", 
                  stringsAsFactors = FALSE)

In [133]:
emp_cols <- employ[c(1:3, 6,10,12)]

In [386]:
head(emp_cols,3)

GEO.id,GEO.id2,GEO.display.label,HC01_VC03,HC01_VC04,HC03_VC04
Id,Id2,Geography,Estimate; EMPLOYMENT STATUS - Population 16 years and over,Estimate; EMPLOYMENT STATUS - Population 16 years and over - In labor force,Percent; EMPLOYMENT STATUS - Population 16 years and over - In labor force
1600000US0100124,100124,"Abbeville city, Alabama",2211,815,36.9
1600000US0100460,100460,"Adamsville city, Alabama",3668,2170,59.2


In [ ]:
### Rename columns

In [134]:
colnames(emp_cols) <- tolower(emp_cols %>% dplyr::slice(1))

In [135]:
emp_cols <- emp_cols[2:nrow(emp_cols), ]

In [136]:
colnames(emp_cols)[4:6] <- c("total_pop_over16", "employ_pop_over16", "per_employ_over16")

In [137]:
summary(emp_cols)

      id                id2             geography         total_pop_over16  
 Length:24256       Length:24256       Length:24256       Length:24256      
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
 employ_pop_over16  per_employ_over16 
 Length:24256       Length:24256      
 Class :character   Class :character  
 Mode  :character   Mode  :character  

In [138]:
emp_cols[ , 4:ncol(emp_cols)] <- sapply(emp_cols[ , 4:ncol(emp_cols)], as.numeric)

In [139]:
summary(emp_cols)

      id                id2             geography         total_pop_over16 
 Length:24256       Length:24256       Length:24256       Min.   :     42  
 Class :character   Class :character   Class :character   1st Qu.:    461  
 Mode  :character   Mode  :character   Mode  :character   Median :   1305  
                                                          Mean   :   7657  
                                                          3rd Qu.:   4359  
                                                          Max.   :6829065  
 employ_pop_over16 per_employ_over16
 Min.   :     18   Min.   : 3.80    
 1st Qu.:    265   1st Qu.:54.90    
 Median :    772   Median :61.90    
 Mean   :   4928   Mean   :60.73    
 3rd Qu.:   2672   3rd Qu.:67.70    
 Max.   :4343671   Max.   :98.50    

In [140]:
emp_cols$state <- as.character(trimws(str_split_fixed(emp_cols$geography, ",", n = 2)[, 2], 
                                      "both"))

In [395]:
summary(emp_cols)

      id                id2             geography         total_pop_over16 
 Length:24256       Length:24256       Length:24256       Min.   :     42  
 Class :character   Class :character   Class :character   1st Qu.:    461  
 Mode  :character   Mode  :character   Mode  :character   Median :   1305  
                                                          Mean   :   7657  
                                                          3rd Qu.:   4359  
                                                          Max.   :6829065  
 employ_pop_over16 per_emply_over16    state          
 Min.   :     18   Min.   : 3.80    Length:24256      
 1st Qu.:    265   1st Qu.:54.90    Class :character  
 Median :    772   Median :61.90    Mode  :character  
 Mean   :   4928   Mean   :60.73                      
 3rd Qu.:   2672   3rd Qu.:67.70                      
 Max.   :4343671   Max.   :98.50                      

###### Load language data

In [141]:
lang <- read.csv("../Preprocessing/Files/Demographics/acs_2015_lang_data.csv", 
                  stringsAsFactors = FALSE)

In [142]:
lang_cols <- lang[c(1:4, 16,18, 28, 30)]

In [143]:
head(lang_cols,3)

GEO.id,GEO.id2,GEO.display.label,HC01_EST_VC01,HC01_EST_VC02,HC02_EST_VC02,HC01_EST_VC03,HC02_EST_VC03
Id,Id2,Geography,Total; Estimate; Population 5 years and over,Total; Estimate; Speak only English,Percent; Estimate; Speak only English,Total; Estimate; Speak a language other than English,Percent; Estimate; Speak a language other than English
1600000US0100100,100100,"Abanda CDP, Alabama",52,52,100,0,0
1600000US0100124,100124,"Abbeville city, Alabama",2578,2530,98.1,48,1.9


In [ ]:
### Rename columns

In [144]:
colnames(lang_cols) <- tolower(lang_cols %>% dplyr::slice(1))

In [145]:
lang_cols <- lang_cols[2:nrow(lang_cols), ]

In [146]:
colnames(lang_cols)[4:8] <- c("total_population", "english_only", "per_english_only", 
                              "english_other", "per_english_other")

In [147]:
summary(lang_cols)

      id                id2             geography         total_population  
 Length:29321       Length:29321       Length:29321       Length:29321      
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
 english_only       per_english_only   english_other      per_english_other 
 Length:29321       Length:29321       Length:29321       Length:29321      
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  

In [148]:
lang_cols[ , 4:ncol(lang_cols)] <- sapply(lang_cols[ , 4:ncol(lang_cols)], as.numeric)

Warning message in lapply(X = X, FUN = FUN, ...):
"NAs introduced by coercion"Warning message in lapply(X = X, FUN = FUN, ...):
"NAs introduced by coercion"

In [149]:
summary(lang_cols)

      id                id2             geography         total_population 
 Length:29321       Length:29321       Length:29321       Min.   :      0  
 Class :character   Class :character   Class :character   1st Qu.:    311  
 Mode  :character   Mode  :character   Mode  :character   Median :   1021  
                                                          Mean   :   7514  
                                                          3rd Qu.:   3837  
                                                          Max.   :7870932  
                                                                           
  english_only     per_english_only english_other     per_english_other
 Min.   :      0   Min.   :  0.00   Min.   :      0   Min.   :  0.00   
 1st Qu.:    276   1st Qu.: 87.80   1st Qu.:      5   1st Qu.:  1.20   
 Median :    912   Median : 95.80   Median :     43   Median :  4.20   
 Mean   :   5664   Mean   : 88.96   Mean   :   1851   Mean   : 11.04   
 3rd Qu.:   3293   3rd Qu.: 98.8

In [151]:
lang_cols$state <- as.character(trimws(str_split_fixed(lang_cols$geography, ",", n = 2)[, 2], 
                                      "both"))

In [152]:
lang_cols <- lang_cols %>% dplyr::rename("total_population_lang" = "total_population") %>%
                mutate(per_english_only = ifelse(total_population_lang == 0, 
                                          0, per_english_only)) %>%
                mutate(per_english_other = ifelse(total_population_lang == 0, 
                                          0, per_english_other)) %>%
                as.data.frame()

In [153]:
summary(lang_cols)

      id                id2             geography         total_population_lang
 Length:29321       Length:29321       Length:29321       Min.   :      0      
 Class :character   Class :character   Class :character   1st Qu.:    311      
 Mode  :character   Mode  :character   Mode  :character   Median :   1021      
                                                          Mean   :   7514      
                                                          3rd Qu.:   3837      
                                                          Max.   :7870932      
  english_only     per_english_only english_other     per_english_other
 Min.   :      0   Min.   :  0.00   Min.   :      0   Min.   :  0.00   
 1st Qu.:    276   1st Qu.: 87.50   1st Qu.:      5   1st Qu.:  1.20   
 Median :    912   Median : 95.70   Median :     43   Median :  4.20   
 Mean   :   5664   Mean   : 88.38   Mean   :   1851   Mean   : 10.97   
 3rd Qu.:   3293   3rd Qu.: 98.70   3rd Qu.:    342   3rd Qu.: 12.20   
 Max.   

###### Rename key variables before joining

In [156]:
#urban_cols2 <- urban_cols2 %>% dplyr::rename("total_area" = "total")
#house_income_cols <- house_income_cols %>% dplyr::rename("house_total_income" = "total_income", 
                                                #         "house_median_income" = "median_income")


Warning message:
"'glue::collapse' is deprecated.
Use 'glue_collapse' instead.
See help("Deprecated") and help("glue-deprecated")."

ERROR: Error: Unknown columns `total_income` and `median_income` 


In [163]:
demo_cols <- age_cols2 %>% left_join(urban_cols2[c("id", "total_area", "urban", "rural", 
                                                   "per_urban",
                                                  "per_rural")], 
                                                  by = "id") %>%
                            left_join(income_cols[c("id",# "total_income", "mean_income", 
                                                #  "income_per_capita", 
                                                   "mean_income_households", 
                                                    "mean_income_per_capita")],
                                      by = "id") %>%
                            left_join(house_income_cols[c("id", "house_total_population", 
                                                        #  "house_total_income", 
                                                          "house_median_income")], 
                                      by = "id") %>%
                            left_join(race_cols[c("id", "total_population_race", 
                                                  "white_population", 
                                                 "black_population", "per_white", 
                                                  "per_black")], by = "id") %>%
                            left_join(emp_cols[c("id", "total_pop_over16", 
                                                 "employ_pop_over16", 
                                                 "per_employ_over16")], by = "id") %>%
                             left_join(lang_cols[c("id", "total_population_lang", 
                                                   "english_only", 
                                                 "english_other", "per_english_only", 
                                                   "per_english_other")], by = "id") %>%
                            
                            as.data.frame()

write.csv(demo_cols, "C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Demographic\\Place\\acs_demo_cols.csv")

Join demographics for cities to US city, state info. 
Creates a features table to use for any knn or variable importance model etc.

In [189]:
usstates_place <- read.csv("../Preprocessing/Files/Demographics/usstates_placetype.csv", 
                  stringsAsFactors = FALSE)

In [194]:
usstates_place$X <- NULL
usstates_place$X.1 <- NULL

In [170]:
demo_cols_csv <- read.csv("../Preprocessing/Files/Demographics/acs_demo_cols.csv", stringsAsFactors = FALSE)

In [195]:
colnames(demo_cols_csv)

'data.frame':	29321 obs. of  36 variables:
 $ rownum                            : chr  "1600000US0100100" "1600000US0100124" "1600000US0100460" "1600000US0100484" ...
 $ id2                               : int  100100 100124 100460 100484 100676 100820 100988 101132 101180 101228 ...
 $ geography                         : chr  "Abanda CDP, Alabama" "Abbeville city, Alabama" "Adamsville city, Alabama" "Addison town, Alabama" ...
 $ totalpopulation                   : int  52 2646 4454 682 293 31905 21406 14787 3912 3727 ...
 $ malepop                           : int  11 1292 2269 331 143 15590 10777 6834 1774 1875 ...
 $ femalepop                         : int  41 1354 2185 351 150 16315 10629 7953 2138 1852 ...
 $ medianage                         : num  60.5 52.5 39.6 42.5 43.6 36.6 33.6 37.9 39.9 30.8 ...
 $ per_male                          : num  21.1 48.8 50.9 48.5 48.8 ...
 $ per_female                        : num  78.8 51.2 49.1 51.5 51.2 ...
 $ total_area                      

In [174]:
demo_cols_csv$X <- NULL
demo_cols_csv$income_per_capita <- NULL
demo_cols_csv$mean_income <- NULL

In [ ]:
#### Fix a few problems/discrepancies with the files and data

In [176]:
demo_cols_csv$total_population_income_households <- income_cols$total_population_income_households
demo_cols_csv$mean_income_households <- income_cols$mean_income_households
demo_cols_csv$total_population_individuals <- income_cols$total_population_individuals
demo_cols_csv$mean_income_per_capita <- income_cols$mean_income_per_capita
demo_cols_csv$house_median_income <- house_income_cols$house_median_income

demo_cols_csv <- demo_cols_csv %>% dplyr::rename("house_total_population" = "total_income")

In [190]:
usstates_place <- usstates_place %>% mutate(City3 = gsub("St.", "St", City2), 
                                            City_state = gsub("St.", "St", City_state),
                                            City_code = gsub("St.", "St", City_code)) %>% 
                as.data.frame()

In [177]:
demo_cols_csv$state <- str_split_fixed(demo_cols_csv$geography, ", ", n = 2)[, 2]
demo_cols_csv$city <- str_split_fixed(demo_cols_csv$geography, ", ", n = 2)[, 1]

write.csv(demo_cols_csv, "C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Demographics\\acs_demo_cols2.csv", row.names = FALSE)

write.csv(usstates_place, "C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Demographics\\usstates_placetype.csv")

In [191]:
str(usstates_place)

'data.frame':	81542 obs. of  29 variables:
 $ X                : int  1 2 3 4 5 6 7 8 9 10 ...
 $ X.1              : int  1 2 3 4 5 6 7 8 9 10 ...
 $ rownum           : int  1 2 3 4 5 6 7 8 9 10 ...
 $ Code             : chr  "AK" "AK" "AK" "AK" ...
 $ State            : chr  "Alaska" "Alaska" "Alaska" "Alaska" ...
 $ Sumlev           : int  40 162 162 162 162 162 162 162 162 162 ...
 $ State_id         : int  2 2 2 2 2 2 2 2 2 2 ...
 $ County           : int  0 0 0 0 0 0 0 0 0 0 ...
 $ Place            : int  0 65 650 870 1090 1200 1420 1860 1970 2080 ...
 $ Cousub           : int  0 0 0 0 0 0 0 0 0 0 ...
 $ Concit           : int  0 0 0 0 0 0 0 0 0 0 ...
 $ Primgeo_flag     : int  0 0 0 0 0 0 0 0 0 0 ...
 $ Funcstat         : chr  "A" "A" "A" "A" ...
 $ City             : chr  "Alaska" "Adak city" "Akhiok city" "Akiak city" ...
 $ Census2010pop    : int  710231 326 71 346 1027 677 219 105 258 324 ...
 $ Estimatesbase2010: int  710249 326 71 346 1027 677 220 171 258 324 ...
 $ Popesti

usstates_place <- usstates_place %>% dplyr::rename("rownum" = "X") %>% as.data.frame()
colnames(demo_cols_csv)[1] <- "rownum"

In [ ]:
### Get unique census population for geographically close locations with the same name

In [196]:
usstates_place2 <- usstates_place %>% group_by(Code, State, State_id, City, City2, City3, Place_type, City_state, City_code) %>%
                                                summarise(avgCensus2010pop = mean(Census2010pop),
                                                            avgPopestimate2015 = mean(Popestimate2015)) %>%
                                        as.data.frame()

In [197]:
usstates_place_unq <- unique(usstates_place2[c("Code", "State", "State_id", "City", 
                              "City2", "City3", "Place_type", "City_state", "City_code", 
                                               "avgCensus2010pop", "avgPopestimate2015")])

In [19]:
str(demo_cols_csv)

'data.frame':	29321 obs. of  34 variables:
 $ rownum                : int  1 2 3 4 5 6 7 8 9 10 ...
 $ id                    : chr  "1600000US0100100" "1600000US0100124" "1600000US0100460" "1600000US0100484" ...
 $ id2                   : int  100100 100124 100460 100484 100676 100820 100988 101132 101180 101228 ...
 $ geography             : chr  "Abanda CDP, Alabama" "Abbeville city, Alabama" "Adamsville city, Alabama" "Addison town, Alabama" ...
 $ totalpopulation       : int  52 2646 4454 682 293 31905 21406 14787 3912 3727 ...
 $ malepop               : int  11 1292 2269 331 143 15590 10777 6834 1774 1875 ...
 $ femalepop             : int  41 1354 2185 351 150 16315 10629 7953 2138 1852 ...
 $ medianage             : num  60.5 52.5 39.6 42.5 43.6 36.6 33.6 37.9 39.9 30.8 ...
 $ per_male              : num  21.1 48.8 50.9 48.5 48.8 ...
 $ per_female            : num  78.8 51.2 49.1 51.5 51.2 ...
 $ total_area            : int  192 2688 4522 758 356 30352 21160 14875 3917 2486 ...


In [ ]:
### Add the additional demographic data to these USA cities

In [198]:
usplace_demo <- usstates_place_unq %>% # filter(County != 0  & Place != 0) %>%
                                    
                                    inner_join(demo_cols_csv, 
                                               by = c("City" = "city", "State" = "state")) %>% 
                                    as.data.frame()

In [35]:
nrow(usplace_demo)

Length  Class   Mode 
     0   NULL   NULL 

write.csv(usplace_demo, "C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Demographics\\usplace_demo.csv")

usplace_demo <- read.csv("C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Demographics\\usplace_demo.csv", stringsAsFactors = FALSE)

In [222]:
str(tweets_latlon)

'data.frame':	5641 obs. of  5 variables:
 $ X         : int  1 2 3 4 5 6 7 8 9 10 ...
 $ City_code : chr  "ABBEVILLE SC" "ABERDEEN MD" "ABERDEEN NC" "ABERDEEN SD" ...
 $ lon       : num  -82.4 -76.2 -79.4 -98.5 -123.8 ...
 $ lat       : num  34.2 39.5 35.1 45.5 47 ...
 $ num_tweets: int  2 5 2 21 46 1 42 133 1 2 ...


In [ ]:
### Total number of tweets for each location 

In [186]:
tweets_latlon <- read.csv("../Preprocessing/Files/Features/tweets_latlon.csv", 
                          stringsAsFactors = FALSE)

In [180]:
tweets_latlon$X <- NULL
usplace_demo$X <- NULL

ERROR: Error in usplace_demo$X <- NULL: object 'usplace_demo' not found


In [199]:
usplace_demo$City_code_cap <- gsub(",", "", toupper(usplace_demo$City_code))

In [ ]:
##### Create a dataframe for all demographic information and tweet counts

In [227]:
features <- usplace_demo %>% inner_join(tweets_latlon, 
                                        by = c("City_code_cap" = "City_code")) %>%
                                 dplyr::rename("location" = "City_code_cap", 
                                               "count" = "num_tweets") %>%
                                 mutate(count_rate = count/totalpopulation * 1000) %>%
                                 as.data.frame()

In [228]:
features$total_area <- ifelse(is.na(features$total_area), 
                              features$totalpopulation, features$total_area)
features$total_pop_over16 <- ifelse(is.na(features$total_pop_over16), 
                              features$totalpopulation, features$total_pop_over16)

In [229]:
avg_per_na <- features %>% group_by(State, add = FALSE) %>%
                                    summarise(med_per_urban = median(per_urban, na.rm = TRUE),
                                                 med_per_rural = median(per_rural, na.rm = TRUE),
                                                    med_per_employ_over16 = 
                                              median(per_employ_over16, na.rm = TRUE)
                                             ) %>%
                            as.data.frame()

In [230]:
features <- features %>% inner_join(avg_per_na, by = "State") %>%
                        mutate(per_urban = ifelse(is.na(per_urban), 
                                           med_per_urban, per_urban)) %>%
                        mutate(per_rural = ifelse(is.na(per_rural), 
                                           med_per_rural, per_rural)) %>%
                        mutate(per_employ_over16 = ifelse(is.na(per_employ_over16), 
                                           med_per_employ_over16, per_employ_over16)) %>%
                        as.data.frame()

In [ ]:
### Add US county information

In [208]:
cities_latlon <- read.csv("../Preprocessing/Files\\Maps\\cities_latlon.csv", stringsAsFactors = FALSE)
cities_latlon$X <- NULL

In [210]:
cities_latlon$location <- paste0(cities_latlon$city, " ",cities_latlon$state)

In [209]:
head(cities_latlon)

lon,lat,city,state,mapid,area,perimeter,feature,name,fips,fips55,pop,state_fips,county,location
-123.8181,46.97494,ABERDEEN,WA,1,0,0,"Population 10,000 to 49,999",Aberdeen,53027,100,16565,53,Grays Harbor County,ABERDEEN WA
-122.2078,48.71663,ACME,WA,2,0,0,"Population less than 10,000",Acme,53073,275,0,53,Whatcom County,ACME WA
-117.8391,48.35600,ADDY,WA,3,0,0,"Population less than 10,000",Addy,53065,380,0,53,Stevens County,ADDY WA
-122.3628,47.33079,ADELAIDE,WA,4,0,0,"Population less than 10,000",Adelaide,53033,415,0,53,King County,ADELAIDE WA
-123.0638,46.62849,ADNA,WA,5,0,0,"Population less than 10,000",Adna,53041,590,0,53,Lewis County,ADNA WA
-118.9817,48.54705,AENEAS,WA,6,0,0,"Population less than 10,000",Aeneas,53047,660,0,53,Okanogan County,AENEAS WA


In [211]:
subset(cities_latlon, is.na(county)) 

,lon,lat,city,state,mapid,area,perimeter,feature,name,fips,fips55,pop,state_fips,county,location
23666,-82.05981,32.39755,METTER,GA,23666,0,0,"Population less than 10,000 County Seat",Metter,13041,51072,3707,13,NA,METTER GA
24044,-80.12637,25.89233,BAL HARBOUR,FL,24044,0,0,"Population less than 10,000",Bal Harbour,12025,3275,3045,12,NA,BAL HARBOUR FL
24051,-80.13057,25.88832,BAY HARBOR ISLANDS,FL,24051,0,0,"Population less than 10,000",Bay Harbor Islands,12025,3975,4703,12,NA,BAY HARBOR ISLANDS FL
24068,-80.18004,25.88322,BISCAYNE PARK,FL,24068,0,0,"Population less than 10,000",Biscayne Park,12025,6600,3068,12,NA,BISCAYNE PARK FL
24100,-80.23669,25.92143,BUNCHE PARK,FL,24100,0,0,"Population less than 10,000",Bunche Park,12025,9500,4388,12,NA,BUNCHE PARK FL
24117,-80.24540,25.94121,CAROL CITY,FL,24117,0,0,"Population less than 10,000",Carol City,12025,10650,0,12,NA,CAROL CITY FL
24148,-80.26780,25.72242,CORAL GABLES,FL,24148,0,0,"Population 10,000 to 49,999",Coral Gables,12025,14250,40091,12,NA,CORAL GABLES FL
24163,-80.34642,25.58155,CUTLER RIDGE,FL,24163,0,0,"Population 10,000 to 49,999",Cutler Ridge,12025,15975,21268,12,NA,CUTLER RIDGE FL
24201,-80.19279,25.85590,EL PORTAL,FL,24201,0,0,"Population less than 10,000",El Portal,12025,20650,2457,12,NA,EL PORTAL FL
24230,-80.47894,25.44846,FLORIDA CITY,FL,24230,0,0,"Population less than 10,000",Florida City,12025,22975,5806,12,NA,FLORIDA CITY FL


In [212]:
usstates_county <- usstates_place %>% dplyr::select(one_of(c("State", "State_id", "Code", 
                                                             "Sumlev", "Place", "County", 
                                                             "City", "City2", "City3"))) %>%
                    filter(Sumlev == 50 & Place == 0) %>%
                    dplyr::rename("County_id" = "County", "County_name" = "City", 
                                  "County2" = "City2", 
                                  "County3" = "City3") %>%
                    as.data.frame()
                        

In [28]:
str(usstates_county)

'data.frame':	3141 obs. of  9 variables:
 $ State2     : chr  "Alaska" "Alaska" "Alaska" "Alaska" ...
 $ State_id2  : int  2 2 2 2 2 2 2 2 2 2 ...
 $ Code2      : chr  "AK" "AK" "AK" "AK" ...
 $ Sumlev     : int  50 50 50 50 50 50 50 50 50 50 ...
 $ Place      : int  0 0 0 0 0 0 0 0 0 0 ...
 $ County_id  : int  13 16 20 50 60 68 70 90 100 105 ...
 $ County_name: chr  "Aleutians East Borough" "Aleutians West Census Area" "Anchorage Municipality" "Bethel Census Area" ...
 $ County2    : chr  "Aleutians East Borough" "Aleutians West Census Area" "Anchorage Municipality" "Bethel Census Area" ...
 $ County3    : chr  "Aleutians East Borough" "Aleutians West Census Area" "Anchorage Municipality" "Bethel Census Area" ...


In [213]:
usstates_place_county <- usstates_place %>%
                            dplyr::left_join(usstates_county[c("State", "State_id", "Code",
                                                          "County_id", 
                                                             "County_name", "County2", 
                                                         "County3")],
                                      by = c("State", "State_id", 
                                             "Code", 
                                             "County" = "County_id")) %>%
                        as.data.frame()

In [214]:
usstates_place_county$County_name <- ifelse(is.na(usstates_place_county$County_name), "None", usstates_place_county$County_name)
usstates_place_county$County_name <- ifelse(is.na(usstates_place_county$County2), "None", usstates_place_county$County2)
usstates_place_county$County_name <- ifelse(is.na(usstates_place_county$County3), "None", usstates_place_county$County3)

write.csv(usstates_place_county, "C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Demographics\\usstates_placetype.csv")

write.csv(features, "C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Features_Demo\\features2.csv")

write.csv(cities_latlon, "C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Maps\\cities_latlon.csv")

In [49]:
str(features)

'data.frame':	5223 obs. of  56 variables:
 $ Code                        : chr  "AK" "AK" "AK" "AK" ...
 $ State                       : chr  "Alaska" "Alaska" "Alaska" "Alaska" ...
 $ State_id                    : int  2 2 2 2 2 2 2 2 2 2 ...
 $ City                        : chr  "Anchorage municipality" "Barrow city" "Fairbanks city" "Homer city" ...
 $ City2                       : chr  "Anchorage" "Barrow" "Fairbanks" "Homer" ...
 $ City3                       : chr  "Anchorage" "Barrow" "Fairbanks" "Homer" ...
 $ Place_type                  : chr  "municipality" "city" "city" "city" ...
 $ City_state                  : chr  "Anchorage, Alaska" "Barrow, Alaska" "Fairbanks, Alaska" "Homer, Alaska" ...
 $ City_code                   : chr  "Anchorage, AK" "Barrow, AK" "Fairbanks, AK" "Homer, AK" ...
 $ avgCensus2010pop            : num  291826 4212 31535 5003 7100 ...
 $ avgPopestimate2015          : num  298018 4439 31638 5510 7636 ...
 $ rownum                      : int  599 611 6